In [984]:
using LinearAlgebra
using PlotlyJS

In [985]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    
    
    
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [986]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end

        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, B, V
        end
        
        # Step 2 of main loop
        # Performing Golub Kahan at matrix B22
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            # If l is equal to k after previous scan, then l = 1 and there is no B11 matrix
            l = (l == k) ? 1 : l 
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80, save_results = false)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    if save_results
        t = min(m, n)
        results = zeros(2, t-1, maxiter)
        
        for k = 1:maxiter
            for j = 2:t
                results[:, j-1, k] = copy(B[j-1:j, j])
            end
            svd_golub_reinsh!(B, P_l, P_r, 1)
        end
        
        B[isapprox.(B, 0.0, atol = 1e-12)] .= 0.0;
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l, results
        else
            return P_l, B, P_r, results
        end
        
    else
        svd_golub_reinsh!(B, P_l, P_r, maxiter)
        B[isapprox.(B, 0.0, atol = 1e-12)] .= 0.0;
        
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l
        else
            return P_l, B, P_r
        end
    end
    
    
    
    
    
end

svd_num (generic function with 3 methods)

## Initial tests

In [987]:
# Example usage 1:
A = randn(8, 6)

8×6 Matrix{Float64}:
 -1.22171    0.233606    1.31613    -0.137042    -1.35723    -0.855544
 -0.374584  -0.184286    0.850247    0.643659     1.58288    -0.501268
  1.71843    0.0438497  -0.479739   -0.681081     0.864115    0.476992
  1.17597   -1.04059     0.666625    0.954249    -0.0820237   0.83031
  1.98986   -0.636702   -0.110931    1.06078     -1.19195     1.34735
  1.82578   -0.628474   -0.21075     1.95819      0.0173218  -0.570325
  0.185577   0.212865    0.0402281   0.00857565   0.353748   -0.314626
  0.753572   0.182925   -0.0187285   0.581051    -1.08265     0.073575

In [988]:
A

8×6 Matrix{Float64}:
 -1.22171    0.233606    1.31613    -0.137042    -1.35723    -0.855544
 -0.374584  -0.184286    0.850247    0.643659     1.58288    -0.501268
  1.71843    0.0438497  -0.479739   -0.681081     0.864115    0.476992
  1.17597   -1.04059     0.666625    0.954249    -0.0820237   0.83031
  1.98986   -0.636702   -0.110931    1.06078     -1.19195     1.34735
  1.82578   -0.628474   -0.21075     1.95819      0.0173218  -0.570325
  0.185577   0.212865    0.0402281   0.00857565   0.353748   -0.314626
  0.753572   0.182925   -0.0187285   0.581051    -1.08265     0.073575

In [989]:
P_l, B, P_r = bidiagonalize(A, true);

In [990]:
B

8×6 Matrix{Float64}:
  3.72345       2.36954      0.0          …   0.0       0.0
  5.55112e-17  -1.55429      1.42162          0.0       5.55112e-17
 -4.44089e-16   2.22045e-16  2.24465          0.0       0.0
 -2.22045e-16  -1.11022e-16  2.22045e-16      1.0651    0.0
 -4.44089e-16  -1.11022e-16  2.22045e-16     -1.85253  -0.314941
 -2.22045e-16  -2.77556e-17  4.44089e-16  …   0.0       0.435647
 -5.55112e-17   1.66533e-16  1.11022e-16      0.0      -2.77556e-17
 -1.11022e-16   2.08167e-17  2.22045e-16      0.0       0.0

In [991]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

3.5752650839881994e-15

In [992]:
U, S, V = svd_num(A);

In [993]:
show(stdout, "text/plain", U)

8×8 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.297407    -0.644204    0.212823    0.0637331  -0.60653    -0.161701   -0.0230215   0.229431
 -0.0845831    0.331811    0.673435    0.291116   -0.108352    0.511174   -0.206617    0.169571
  0.258234     0.502981   -0.329413   -0.018465   -0.718221   -0.0969591  -0.203197    0.0702922
  0.396692    -0.083899    0.217689    0.638727   -0.0668031  -0.33453     0.107497   -0.502302
  0.607447    -0.300375   -0.225977    0.250003    0.0782908   0.337122    0.121619    0.540966
  0.516655    -0.0131319   0.529095   -0.577674    0.0151884  -0.324292   -0.0420845   0.109903
 -0.00741236   0.11838     0.0904436  -0.162746   -0.257434    0.247589    0.897945   -0.131864
  0.210633    -0.332359   -0.0781197  -0.283164   -0.165666    0.557787   -0.284724   -0.58245

In [994]:
show(stdout, "text/plain", S)

8×6 Matrix{Float64}:
 4.51316   0.0       0.0           0.0       0.0      0.0
 0.0      -2.86496  -2.90233e-12   0.0       0.0      0.0
 0.0       0.0       2.46876       0.0       0.0      0.0
 0.0       0.0       0.0          -1.51795   0.0      0.0
 0.0       0.0       0.0           0.0      -1.00833  0.0
 0.0       0.0       0.0           0.0       0.0      0.426657
 0.0       0.0       0.0           0.0       0.0      0.0
 0.0       0.0       0.0           0.0       0.0      0.0

In [995]:
show(stdout, "text/plain", V')

6×6 adjoint(::Matrix{Float64}) with eltype Float64:
  0.800917   -0.250351   -0.111517   0.43592   -0.10755     0.286055
 -0.201834   -0.0215103   0.284781   0.2294    -0.907717   -0.0335622
 -0.140991   -0.202141    0.435238   0.643286   0.352282   -0.460335
  0.17678     0.386567   -0.565777   0.152309  -0.162688   -0.670482
  0.515973    0.226311    0.604472  -0.449377  -0.0315193  -0.337987
 -0.0209135   0.833877    0.177602   0.344084   0.113666    0.375974

In [996]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.4098481629358565e-15

In [997]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

2.119969625361317e-15

In [998]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.031065067959966e-15

In [999]:
# Example usage 2:
A = randn(9, 7)

9×7 Matrix{Float64}:
  1.38094   -0.368812  -0.340407   …   0.451576   1.74632     0.808075
 -1.81207    1.3306    -0.670393      -0.482116  -0.961233    0.169656
 -0.293423   0.879646  -0.966709      -0.454261   0.0892117  -0.0306507
 -2.64274   -0.74185   -0.926277       0.852429  -0.675763    0.800132
 -2.30382   -0.243845  -0.868822      -1.39812    0.155706    0.0887309
  0.466675   0.522945   0.0185426  …   2.17915    0.505091   -0.244521
  1.80123    0.568044  -0.36094       -0.971073   0.517981   -0.38294
  0.225     -0.120363  -0.2793         0.411145   1.15998     0.46907
  0.777079  -0.74897   -0.65994        0.182945  -1.07299    -3.19768

In [1000]:
U, S, V = svd_num(A);

In [1001]:
show(stdout, "text/plain", U)

9×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.421705   -0.374599   -0.0672983  -0.0102742  -0.524783   -0.411663    0.136581  -0.436328     0.15266
  0.133308   -0.191121    0.735626   -0.240872    0.26986    -0.205612    0.137278   0.00116344   0.461073
 -0.0963914  -0.139302    0.451922   -0.287055   -0.151145   -0.0662597  -0.296127   0.0376553   -0.753874
  0.562527   -0.155104    0.0549484   0.454208   -0.277434   -0.300151   -0.517894   0.0618876    0.105252
  0.483949   -0.0293591   0.0620951  -0.315575   -0.510065    0.538894    0.207479  -0.253104     0.0616902
 -0.338803   -0.200847    0.326439    0.548755    0.0590302   0.589284   -0.179173  -0.224505     0.0627092
 -0.266742    0.133083   -0.112183   -0.460765   -0.0872938   0.155305   -0.701593   0.0503822    0.399671
 -0.181766   -0.28015     0.0402178   0.0471726  -0.369673    0.128382    0.17819    0.828511     0.115376
 -0.15963     0.80143     0.34993     0.194732   -0.37952    -0.124546    0.086367   0.0305

In [1002]:
show(stdout, "text/plain", S)

9×7 Matrix{Float64}:
 -5.86502  0.0       0.0          0.0       0.0      0.0        0.0
  0.0      3.94279   2.97959e-12  0.0       0.0      0.0        0.0
  0.0      0.0      -3.48044      0.0       0.0      0.0        0.0
  0.0      0.0       0.0          3.00258   0.0      0.0        0.0
  0.0      0.0       0.0          0.0      -2.03745  0.0        0.0
  0.0      0.0       0.0          0.0       0.0      0.710442   0.0
  0.0      0.0       0.0          0.0       0.0      0.0       -0.882476
  0.0      0.0       0.0          0.0       0.0      0.0        0.0
  0.0      0.0       0.0          0.0       0.0      0.0        0.0

In [1003]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.716229   0.0808968   0.0934434  -0.597792    0.169007    0.260325   -0.134358
  0.26711   -0.18069     0.0144757   0.254966   -0.162418   -0.405816   -0.799825
  0.464185  -0.340568    0.346966    0.605103   -0.07773     0.297039    0.296203
 -0.126134  -0.318232    0.110148    0.0622537   0.922057   -0.0945229  -0.0876727
 -0.11345   -0.520202   -0.603838   -0.0391748  -0.0835354   0.548851   -0.205298
 -0.194105   0.542517    0.215435    0.266692    0.137036    0.591489   -0.426408
  0.365709   0.422627   -0.66896     0.367359    0.251134   -0.138854    0.151109

In [1004]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

8.857680597909574e-14

In [1005]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

2.2856682086432482e-15

In [1006]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.3499413199454106e-15

In [1007]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
 -2.12984    0.481909   4.10877   …  -1.88848   -0.567348   0.146
  0.481909  -0.881156   0.706425     -0.695603  -0.196634   0.919064
  4.10877    0.706425   2.74779      -1.56039   -1.04075   -0.953511
 -2.80514   -0.937981  -1.66809       0.484343   2.36318    0.0498807
 -1.88848   -0.695603  -1.56039      -0.37572    1.19283    0.106419
 -0.567348  -0.196634  -1.04075   …   1.19283   -1.82667    0.528821
  0.146      0.919064  -0.953511      0.106419   0.528821  -0.111033

In [1008]:
U, S, V = svd_num(A);

In [1009]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.441843   -0.81578    -0.27879    -0.110756    0.094577    0.180332    0.0884472
  0.139866   -0.074329    0.0127331  -0.491252   -0.0779932  -0.835034   -0.173371
  0.613895    0.359121    0.232223   -0.0764826   0.368015   -0.0283457   0.546031
 -0.518876   -0.282444    0.260603   -0.257498   -0.127795   -0.0410007   0.70623
 -0.274956   -0.241316    0.260209    0.248623    0.815468   -0.208071   -0.16841
 -0.242958    0.233553   -0.825874   -0.200443    0.346616   -0.0316036   0.207457
 -0.0669549   0.0863452   0.224713   -0.756478    0.219262    0.472675   -0.306282

In [1010]:
show(stdout, "text/plain", S)

7×7 Matrix{Float64}:
 8.49065  2.02175e-12   0.0      0.0       0.0        0.0           0.0
 0.0      5.27756       0.0      0.0       0.0        0.0           0.0
 0.0      0.0          -2.98793  0.0       0.0        0.0           0.0
 0.0      0.0           0.0      0.532897  0.0        0.0           0.0
 0.0      0.0           0.0      0.0       0.772682   1.07256e-12   0.0
 0.0      0.0           0.0      0.0       0.0       -1.70831       0.0
 0.0      0.0           0.0      0.0       0.0        0.0          -1.65224

In [1011]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.441843    0.139866    0.613895   -0.518876   -0.274956  -0.242958   -0.0669549
  0.81578     0.074329   -0.359121    0.282444    0.241316  -0.233553   -0.0863452
 -0.27879     0.0127331   0.232223    0.260603    0.260209  -0.825874    0.224713
 -0.110756   -0.491252   -0.0764826  -0.257498    0.248623  -0.200443   -0.756478
 -0.094577    0.0779932  -0.368015    0.127795   -0.815468  -0.346616   -0.219262
  0.180332   -0.835034   -0.0283457  -0.0410007  -0.208071  -0.0316036   0.472675
 -0.0884472   0.173371   -0.546031   -0.70623     0.16841   -0.207457    0.306282

In [1012]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

8.988947674057197e-14

In [1013]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

3.2087602712912088e-15

In [1014]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.895512949811143e-15

## Cubic convergence depiction

### Square 7 x 7

In [1015]:
A = randn(7, 7)

7×7 Matrix{Float64}:
 -0.580772   0.147533   -0.106461    …  -1.3489    -0.598577  -0.840475
  0.152986   0.137215    0.497523       -1.33759   -0.33129   -0.0204833
  2.03717    2.11836     1.10054        -0.276066  -1.17295   -0.481577
 -0.256688  -0.270827   -2.08137         1.14638    1.2368    -0.328911
 -0.182723  -0.535633   -4.38952e-5     -0.818669   1.33274   -1.90949
 -0.396727  -0.660086   -0.0277177   …  -0.342824   0.421169  -0.537468
 -0.930788   0.0438853   1.04414        -0.716094  -0.404759   0.13166

In [1016]:
U, S, V, results = svd_num(A, 20, true);

In [1017]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.929529530994163e-14

In [1018]:
function plot_convergence_error(error_vectors, labels)
    iterations = 1:length(error_vectors[1])
    traces = Vector{GenericTrace{Dict{Symbol, Any}}}()
    
    for (i, error_vector) in enumerate(error_vectors)
        trace = scatter(
            x = iterations,
            y = error_vector,
            mode = "lines+markers",
            marker_color = i,
            line_shape = "linear",
            name = labels[i]
        )
        push!(traces, trace)
    end
    
    layout = Layout(
        title = "Value of element above diagonal σ_i",
        xaxis_title = "Iteration",
        yaxis_title = "Error",
        yaxis_type = "log",
        yaxis_tickformat = "e",
        showlegend = true
    )
    
    plot(traces, layout)
end

error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [1019]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.38158  -2.88769  -2.87955  -2.98102  -3.05232  -3.08695  -3.10288  -3.10819  -3.11  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073  -3.11073

In [1020]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.39699  -2.59299  -2.67706  -2.55824  -2.49206  -2.46262  -2.44959  -2.44532  -2.44388  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433  -2.4433

In [1021]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.97484  1.19014  1.15156  1.20436  1.25184  1.28478  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855  1.31855

In [1022]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -0.582516  0.715327  1.21527  1.17608  1.13162  1.1026  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435  1.07435

In [1023]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.20527  -0.431724  -0.237094  -0.233608  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557  -0.233557

In [1024]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 0.440102  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363  0.440363

In [1025]:
S

7×7 Matrix{Float64}:
 4.3516   0.0       0.0     0.0      0.0       0.0       0.0
 0.0     -3.11073   0.0     0.0      0.0       0.0       0.0
 0.0      0.0      -2.4433  0.0      0.0       0.0       0.0
 0.0      0.0       0.0     1.31855  0.0       0.0       0.0
 0.0      0.0       0.0     0.0      1.07435   0.0       0.0
 0.0      0.0       0.0     0.0      0.0      -0.233557  0.0
 0.0      0.0       0.0     0.0      0.0       0.0       0.440363

### Rectangular 9 x 7

In [1026]:
A = randn(9, 7)

9×7 Matrix{Float64}:
  0.543105   -0.278273    1.29883    …   0.563661    0.0556868  -0.0439231
  0.891825   -0.759537   -0.407143      -0.437829   -1.17603     0.991998
  1.0386      0.0997983  -0.180994      -0.107601   -1.68721    -1.32035
 -1.24854    -0.223681    2.46235        0.469211    0.420167   -1.30332
 -1.14996     0.158933   -0.0515083      0.323799   -1.85297     0.43316
 -0.353496    1.79451     0.534322   …  -0.0453961   0.232003    0.028331
  0.875255   -0.110205    0.376168       2.85106     0.678114    2.2774
  0.0281053   0.0143283  -1.60362       -0.206587   -0.557265   -0.163481
  0.519704    0.461953    0.424803      -1.31291     2.156       0.848169

In [1027]:
U, S, V, results = svd_num(A, 20, true);

In [1028]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.761596244971605e-15

In [1029]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [1030]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.74954  -1.36689  3.18962  4.46064  4.38672  4.14458  4.02487  3.97226  3.95406  3.94674  3.94258  3.94221  3.94217  3.94217  3.94217  3.94217  3.94217  3.94217  3.94217  3.94217

In [1031]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.43981  -1.99232  -1.07638  -1.09722  1.41046  -1.84408  -3.22263  -3.4605  -3.47788  -3.47751  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694  -3.47694

In [1032]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.02716  2.65288  -1.92991  -1.28982  0.954642  -0.724644  -0.413921  -0.385305  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309  -0.383309

In [1033]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.92383  -2.21839  -2.07921  -1.64906  -1.594  -1.59402  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403  -1.59403

In [1034]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.64704  -1.59843  -1.72108  -2.171  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461  -2.2461

In [1035]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.0899  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115  -2.16115

In [1036]:
S

9×7 Matrix{Float64}:
 -4.95178  0.0       0.0       0.0        0.0       0.0      0.0
  0.0      3.94217   0.0       0.0        0.0       0.0      0.0
  0.0      0.0      -3.47694   0.0        0.0       0.0      0.0
  0.0      0.0       0.0      -0.383309   0.0       0.0      0.0
  0.0      0.0       0.0       0.0       -1.59403   0.0      0.0
  0.0      0.0       0.0       0.0        0.0      -2.2461   0.0
  0.0      0.0       0.0       0.0        0.0       0.0     -2.16115
  0.0      0.0       0.0       0.0        0.0       0.0      0.0
  0.0      0.0       0.0       0.0        0.0       0.0      0.0

### Rectangular 7 x 9

In [1037]:
A = randn(7, 9)

7×9 Matrix{Float64}:
  1.15242   -0.613381    0.484037  …   1.00148   -0.135562   0.599141
  0.549875   0.0444981   1.07999      -1.87375    0.577872  -2.08652
 -0.337832  -0.162021    0.182423      0.935028   0.8517     1.15197
 -0.143251  -0.279768    0.538681      0.58682   -0.493972  -0.213476
  0.397573   1.48991    -0.727834     -2.30472   -0.222902  -0.48914
 -0.990749   0.501166    0.82382   …  -0.670944  -1.69145   -0.4325
  1.11269   -1.18263    -0.178186      1.54898    1.81868    0.697078

In [1038]:
U, S, V, results = svd_num(A, 20, true);

In [1039]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.4167541439889405e-14

In [1040]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [1041]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.88582  1.8687  2.29709  -2.94626  -3.8047  -3.947  -3.9644  -3.96664  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668  -3.96668

In [1042]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.63002  2.30644  1.48609  -1.03467  -0.739596  -0.69856  -0.693378  -0.75662  1.25225  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351  0.685351

In [1043]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.03909  1.73981  -1.65123  1.58219  -2.16355  2.72333  2.92947  2.72896  -1.64884  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271  -3.01271

In [1044]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.85655  -2.01495  2.31067  -2.62562  2.08068  -1.6867  -1.57274  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463  -1.5463

In [1045]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.72534  -1.88263  -2.0453  -2.0896  -2.08721  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724  -2.08724

In [1046]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.28272  2.529  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992  2.52992

In [1047]:
S

7×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -5.21056   0.0      0.0        0.0       0.0      0.0      0.0      0.0  0.0
  0.0      -3.96668  0.0        0.0       0.0      0.0      0.0      0.0  0.0
  0.0       0.0      0.685351   0.0       0.0      0.0      0.0      0.0  0.0
  0.0       0.0      0.0       -3.01271   0.0      0.0      0.0      0.0  0.0
  0.0       0.0      0.0        0.0      -1.5463   0.0      0.0      0.0  0.0
  0.0       0.0      0.0        0.0       0.0     -2.08724  0.0      0.0  0.0
  0.0       0.0      0.0        0.0       0.0      0.0      2.52992  0.0  0.0